In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_red_columns(image, pixel_threshold=30, debug=False):
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # HSV thresholds for red
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])

    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)

    # Count red pixels in each column
    red_pixel_counts_per_column = np.sum(red_mask > 0, axis=0)

    # Threshold to avoid counting faint/noisy columns
    red_columns = red_pixel_counts_per_column > pixel_threshold
    num_red_columns = np.sum(red_columns)

    if debug:
        # Plot red pixel count per column
        plt.figure(figsize=(12, 3))
        plt.plot(red_pixel_counts_per_column, color='red')
        plt.title("Red Pixel Count Per Column")
        plt.xlabel("Column Index")
        plt.ylabel("Red Pixels")
        plt.grid(True)
        plt.show()

    return num_red_columns, red_mask

# === STEP 1: Calibration from Reference Image ===
reference_image_path = 'C:/Users/kaurn/Desktop/500 45.png'  # Replace with your reference image
known_temperature = 45.0  # Red mercury section = 45°C

ref_img = cv2.imread(reference_image_path)
if ref_img is None:
    raise ValueError(f" Could not load reference image: {reference_image_path}")

ref_red_columns, ref_mask = detect_red_columns(ref_img, debug=True)
image_width = ref_img.shape[1]
TPP = known_temperature / ref_red_columns

print(f"\n REFERENCE CALIBRATION")
print(f" Total image width             : {image_width} pixels")
print(f" Red columns in reference     : {ref_red_columns}")
print(f" Known red temperature        : {known_temperature} °C")
print(f" Calculated TPP               : {TPP:.4f} °C per red column")

# === Visual Confirmation for Reference Image ===
plt.figure(figsize=(10, 3))
plt.imshow(cv2.cvtColor(ref_img, cv2.COLOR_BGR2RGB))
plt.title("Reference Image")
plt.axis('off')
plt.show()

plt.figure(figsize=(10, 3))
plt.imshow(ref_mask, cmap='gray')
plt.title("Detected Red Area (Reference)")
plt.axis('off')
plt.show()

# === STEP 2: Analyze Test Images ===
test_image_paths = [
    '500 70.jpg',
    
]

for test_path in test_image_paths:
    test_img = cv2.imread(test_path)
    if test_img is None:
        print(f"\n Could not load test image: {test_path}")
        continue

    test_red_columns, test_mask = detect_red_columns(test_img, debug=True)
    temperature = test_red_columns * TPP

    print(f"\n ANALYSIS FOR: {test_path}")
    print(f" Red columns detected         : {test_red_columns}")
    print(f" Estimated Temperature        : {temperature:.2f} °C")

    # Visual display
    plt.figure(figsize=(10, 3))
    plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
    plt.title(f"{test_path} - Estimated Temp: {temperature:.2f} °C")
    plt.axis('off')
    plt.show()

    plt.figure(figsize=(10, 3))
    plt.imshow(test_mask, cmap='gray')
    plt.title("Detected Red Area (Test Image)")
    plt.axis('off')
    plt.show()